In [2]:
# tester.ipynb
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))


from src.analyzer import analyze_speech

# Quick analysis
result = analyze_speech("../data/l2arctic_spontaneous/L2A_020.wav")
# result = analyze_speech("../samples/sample4.flac")

# Access results
print(f"Score: {result['verdict']['fluency_score']}")
print(f"Fillers detected: {len(result['filler_events'])}")

result

Analyzing audio: ../data/l2arctic_spontaneous/L2A_020.wav
Context: conversational

[1/5] Transcribing with Whisper (verbatim)...
  Duration: 74.98s
  Words: 121

[2/5] Marking filler words...
  Marked: 4 filler words
  Content words: 117

[3/5] Aligning words with WhisperX...
  Aligned: 121 words

[4/5] Detecting subtle fillers with Wav2Vec2...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Total events: 11

[5/5] Calculating fluency score...

✓ Fluency Score: 76/100
✓ Readiness: borderline
Score: 76
Fillers detected: 11


{'verdict': {'fluency_score': 76, 'readiness': 'borderline'},
 'benchmarking': {'percentile': 65,
  'target_score': 80,
  'score_gap': 4,
  'estimated_guided_practice_hours': 2.4},
 'normalized_metrics': {'wpm': 93.6249666577754,
  'fillers_per_min': np.float64(3.5209389170445453),
  'stutters_per_min': 0.8002133902373966,
  'long_pauses_per_min': 0.0,
  'very_long_pauses_per_min': 0.0,
  'pause_time_ratio': np.float64(0.00880234729261134),
  'pause_variability': 0.0,
  'vocab_richness': 0.5641025641025641,
  'repetition_ratio': np.float64(0.07692307692307693)},
 'opinions': {'primary_issues': [{'issue': 'filler_dependency',
    'severity': 'high',
    'root_cause': 'Fillers replace silent planning pauses.',
    'score_impact': 14},
   {'issue': 'delivery_pacing',
    'severity': 'medium',
    'root_cause': 'Speech rate deviates from optimal clarity range.',
    'score_impact': 6}],
  'action_plan': [{'priority': 1,
    'focus': 'filler_dependency',
    'instruction': 'Replace filler w